# Bayesian Networks Essays

## Probability Calculus

Some useful rules derived from [Kolgomorov Axioms](https://en.wikipedia.org/wiki/Probability_axioms) for random variables:


Conditional Probability:
    P(A,B)=P(B|A)P(A)=P(A|B)P(B)


Chain Rule: 
    P(A1,A2…An)=P(A1)P(A2|A1)…P(An|A1,A2…An−1)
    

In [35]:
%load_ext pep8_magic

import numpy as np
import re

In [144]:
class BayesianNetework():
    """
    
    """
    prior_list = set([])
    values = {}
    verbose = False
    
    def __init__(self, verbose: bool=False):
        self.verbose = verbose
    
    def display(self, of: str):
        if self.verbose:
            print('P(%s)=%s' % (of, self.values[of]))
    
    def P(self, of: str):
        """
        
        """
        self.display(of)
        return self.values[of]
    
    def prior(
        self, of: str=None, value: float=None
    ):
        """
        P(C): the prior probability (what we know before the evidence)
        """
        if not value is None:
            self.values[of] = value
            self.prior_list |= {of}
            
        return self.P(of)
    
    def likelihood(
        self, of: str=None, given: str=None, value: float=None
    ):
        """
        P(+|C): the likelihood of the data given the hypothesis
        """
        term = '%s|%s' % (of, given)
        
        if not value is None:
            self.values[term] = value
            
        return self.P(term)
 
    def p_joint(self, A: str, B: str):
        """
        
        """
        term = '%s,%s' % (A, B)
        self.values[term] = self.P(A) * self.P(B)
        
        self.display(term)
        return self.values[term]
    
    def p_total(self):
        """
        Total Probability
        
        P(A|C) = ∑i P(A|C,Bi) * P(Bi|C)
        """
        pass
    
    def p_marginal(self, of: str):
        """
        Bi∩Bj=∅,∑iBi=Ω:
        
        P(A) = ∑iP(A,Bi)
        
        """
        verbose = self.verbose
        self.verbose = False
        result = sum([self.p_joint('%s|%s' % (of, b), b) for b in self.prior_list])
        
        self.verbose = verbose
        return result
    
    def bayes_rules(self, of: [str], given: [str]):
        """
        
        P(A|B,C) = (P(B|A,C)*P(A|C))/P(B|C)
        
        Example:
        
        P(C|+) = (P(+|C)*P(C))/P(+)
        
        """
        verbose = self.verbose
        self.verbose = False
        
        P = self.P
        
        _of = '.'.join(of)
        _given = ','.join(given)
        
        _likelihood = '%s|%s' % (_given, _of)
        _prior = _of
        _evidence = _given
        
        result = (P(_likelihood) * P(_prior))/P(_evidence)
        self.verbose = verbose
        
        return result
    
    def evidence(self, of: str):
        """
        P(+): the evidence (the marginal probability of the test is positive)
        
        """
        self.values[of] = self.p_marginal(of)
        return self.P(of)
    
    def proportional_posterior(self, of: [str], given: [str]):
        """
        Posterior probability ∝ Likelihood × Prior probability
        
        """
        verbose = self.verbose
        self.verbose = False
        
        P = self.P
        p = {}
        
        _of = '.'.join(of)
        
        for i, _prior in enumerate(self.prior_list):
            _p_likelihood = []
            for _given in given:
                _likelihood = '%s|%s' % (_given, _prior)
                _p_likelihood.append(P(_likelihood))
                
            p[_prior] = np.prod(np.array(_p_likelihood)) * P(_prior)
        
        self.verbose = verbose
        
        # sum(p) == proportional constant
        return p[_of]/sum(p.values())
        
    
    def posterior(self, of: [str], given: [str]):
        """
        P(C|+): the posterior probability, the new belief after the evidence
        is processed, using Bayes’ rule.
        
        The posterior probability can be written in the memorable form as

        Posterior probability ∝ Likelihood × Prior probability
        
        """
        
        if isinstance(of, str):
            of = [of]
            
        if isinstance(given, str):
            given = [given]
        
        _of = '.'.join(of)
        _given = ','.join(given)
        
        term = '%s|%s' % (_of, _given)
        
        if _given in self.values:
            self.values[term] = self.bayes_rules(of=of, given=given)
        else:
            self.values[term] = self.proportional_posterior(
                of=of, given=given
            )
            
        self.display(term)
        
        return self.values[term]
    

## Cancer test

### Definition

$
P(C) = 0.01\\
P(\neg C) = 0.99\\
P(+|C) = 0.9\\
P(-|C) = 0.1\\
P(+|\neg C) = 0.2\\
P(-|\neg C) = 0.8
$

In [151]:
bnet = BayesianNetework(verbose=True)

# Prior Probability
print('\nPrior Probability')
P = bnet.prior
P('C', 0.01)
P('!C', 1-P('C'))

# likelihood of the data given the hypothesis
print('\nlikelihood of the data given the hypothesis')
P = bnet.likelihood
P(of='+', given='C', value=0.9)
P(of='-', given='C', value=0.1)
P(of='+', given='!C', value=0.2)
P(of='-', given='!C', value=0.8)

print('\nEvidence')
P = bnet.evidence
P('+')
P('-')

print('\nThe posterior probability')
P = bnet.posterior
P(of='C', given='+')
P(of='C', given='-')


Prior Probability
P(C)=0.01
P(C)=0.01
P(!C)=0.99

likelihood of the data given the hypothesis
P(+|C)=0.9
P(-|C)=0.1
P(+|!C)=0.2
P(-|!C)=0.8

Evidence
P(+)=0.20700000000000002
P(-)=0.793

The posterior probability
P(C|+)=0.043478260869565216
P(C|-)=0.0012610340479192938


0.0012610340479192938

### Two Cancer Test

In [152]:
print('\nThe posterior probability')
P = bnet.posterior
P(of='C', given=['+', '+'])
P(of='!C', given=['+', '+'])
P(of='C', given=['+', '-'])
# P(of=['C', '+'], given=['+'])


The posterior probability
P(C|+,+)=0.169811320755
P(!C|+,+)=0.830188679245
P(C|+,-)=0.00564971751412


0.005649717514124292

## References

Bayesian Networks - João Neto - December 2013
http://www.di.fc.ul.pt/~jpn/r/bayesnets/bayesnets.html
